In [1]:
import pandas as pd
import numpy as np
import sklearn
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("/Users/Triveni/Desktop/dataScience/data/train.csv")

In [3]:
catVars = [col for col in list(train) if train[col].dtype=="object"]
trainPlusDummies = pd.get_dummies(data=train,columns=catVars)
trainPlusDummies.dropna(inplace=True)

In [4]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score

In [5]:
X = trainPlusDummies.drop(columns=["SalePrice","Id"])
Y = trainPlusDummies.SalePrice
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(X,Y)

In [6]:
lasso = LassoCV()
rfe = RFECV(estimator=lasso,cv=5)
rfe.fit(x_train, y_train)
features = X.columns[rfe.get_support()]

In [7]:
X[features].head()

,MSSubClass,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,TotalBsmtSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,ScreenPorch,PoolArea
0,60,2003,2003,196.0,706,856,1710,548,0,61,0,0
1,20,1976,1976,0.0,978,1262,1262,460,298,0,0,0
2,60,2001,2002,162.0,486,920,1786,608,0,42,0,0
3,70,1915,1970,0.0,216,756,1717,642,0,35,0,0
4,60,2000,2000,350.0,655,1145,2198,836,192,84,0,0


In [8]:
x_train_new = rfe.transform(x_train)
x_test_new = rfe.transform(x_test)

In [9]:
linearModel = sklearn.linear_model.LinearRegression()
linearModel.fit(x_train_new,y_train)
scores = cross_val_score(linearModel,x_test_new,y_test,cv=10)

In [10]:
scores

array([ 0.67695856,  0.26060738,  0.80832826,  0.87770273, -0.85597834,
        0.66022491,  0.85796738,  0.65643137,  0.79288861,  0.81544444])